## Analysing the peak results from a hot-deformation experiment

This notebook is used to extract peak results calculated from Continuous-Peak-Fit, to calculate the elastic strain partitioning of two phase materials during hot-deformation.

In [ ]:
import matplotlib.pyplot as plt

import continuous_peak_fit_analysis_functions as analysis
import continuous_peak_fit_deformation_functions as deformation
import continuous_peak_fit_sxrd_functions as sxrd

%load_ext autoreload
%autoreload 2

In [ ]:
pwd

## Load YAML file

The file paths and user inputs for the analysis of the Diamond 2017, 2021, 2022, and DESY 2020, 2021 experiments are included in the `yaml` configuration files, to record the inputs of the analysis.

In this case, there are 2x `yaml` files required to analyse the data. 

The **first** yaml file contains information about the .fit files, which can also be used to directly extract the intensities for calculating crystallographic texture.

The **second** yaml file contains additional information about the deformation setup.

The information in the `yaml` files can be accessed like this...

In [ ]:
# to load the Diamond 2017, Diamond 2021, DESY 2020 or DESY 2021 experiment analysis
config_path_texture = "yaml/texture/diamond/config_diamond_2022_038.yaml"
config_path_deformation = "yaml/deformation/diamond/config_diamond_2022_038.yaml"

config_texture = analysis.get_config(config_path_texture)
config_deformation = analysis.get_config(config_path_deformation)

## Extracting the single peak profile data from fit files

The single peak profile data (such as the peak position, peak intensity, half-width and pseudo-Voigt weighting) obtained from the Fourier peak analysis in Continuous-Peak-Fit can be extracted from the .fit files using the `read_fit_results` function.

The user inputs required to extract the intensity data are taken directly from the **first** `yaml` configuration file, using the `extract_intensity_input` function. The user inputs include;

- Input and output file paths
- A list of lattice plane peak labels
- Azimuthal data resolution
- Image numbers

*Note, `image_numbers` is returned as retaining information about the different analysis blocks, which is useful to see each stage of the data load, but `image_numbers_sorted` is a single list of all the image_numbers, which is useful for plotting the data.*

In [ ]:
experiment_number, input_fit_path, peak_label, data_resolution, \
image_numbers, image_numbers_sorted = deformation.extract_analysis_input(config_path_texture)

In [ ]:
peak_position, peak_intensity, peak_halfwidth, peak_PV_weight = analysis.read_fit_results(experiment_number, 
                                                                input_fit_path, peak_label, data_resolution, 
                                                                image_numbers)

To make sense of the data and look at the behaviour over time, it makes sense to restructure it. We do this by writing the peak results to new arrays. This will then allow us to plot the behaviour of single lattice plane peaks over time.

In [ ]:
peak_position_time, peak_intensity_time, \
peak_halfwidth_time, peak_PV_weight_time \
= deformation.restructure_fit_results(image_numbers, peak_label, data_resolution, 
                                      peak_position, peak_intensity, peak_halfwidth, peak_PV_weight)

The new arrays are structured as nested dictionaries, and the data can be accessed by first defining the lattice plane, followed by the azimuthal angle. For example, using this command:

`print(peak_position_time["10-10"][359])`

the position of the {10-10} lattice plane at an azimuthal angle of ${359^\circ}$ is recorded for each of the diffraction pattern image numbers.

## Plot commands

The following plot commands will be used for subsequent plots.

In [ ]:
plt.rc('xtick',labelsize=20)
plt.rc('ytick',labelsize=20)
plt.rc('legend',fontsize=20)
plt.rc('axes',linewidth=2)
plt.rc('xtick.major',width=2,size=10)
plt.rc('xtick.minor', width=2, size=5)
plt.rc('ytick.major',width=2,size=10)
plt.rc('ytick.minor',width=2,size=5)

## Load and Synchronise the thermomechanical data

The thermomechanical test data must be loaded and correctly callibrated to the corresponding synchrotron diffraction pattern image number, to make sense of the effect of hot-deformation on the lattice plane peak changes observed in the synchrotron data.

The parameters for the thermomechanical equipment can be loaded from the **second** yaml file. 

These inputs also include some information about the synchrotron data, such as the acquisition frequency, so that the thermomechanical test data can be adjusted to the correct frequency. 

*Note, if this is the first time loading the data then it is possible to leave out the `start_deformation` and `end_deformation` image numbers in the yaml file, and instead use some initial guesses for `start_deformation_initial` and `end_deformation_initial` which you know will approximately cover the entire deformation region. Then, when you have fully checked the data you can go back and rerun this analysis with the correct definition for the start and end image numbers.*

In [ ]:
thermomechanical_equipment, thermomechanical_file_path, start_deformation, \
end_deformation, number_of_frames, acquisition_frequency_sxrd, \
minimum_stress, deform_sequence, filter_equipment, output_file_path \
= deformation.extract_thermomechanical_input(config_path_deformation)

The next cell uses the `plot_thermomechanical_data` command to calculate the true stress-strain behaviour measured on the thermomechanical equipment, which is slightly adjusted to match the acquisition frequency of the synchrotron diffraction pattern images.

There are currently two types of data that can be loaded:

1. Instron Electro-thermal mechanical tester (ETMT)
2. DIL 805 A/D/T Compression Dilatometer (Dilatometer)

In both cases the data has been reduced to only cover the hot-deformation regime.

For data from the **ETMT**, a separate notebook [sxrd-thermomechanical-test-analysis](https://github.com/LightForm-group/sxrd-thermomechanical-test-analysis) is first needed to calculate the true stress versus true strain behaviour, which is based on a resistance method to calculate the plastic strain. This creates a text file with the converted true stress and true strain, along with other measured parameters such as temperature. This ETMT data has already been reduced, using the notebook, to start at the point of applied load.

For data from the **Dilatometer**, the true stress and true strain data is included in an ascii text file, which can be saved using the accompanying Dilatometer software. In the case of the Dilatometer, the deformation does not start when the deformation file begins recording, and so a stress signal (typically > 1 MPa for more than 4 points) is used to signal the start of deformation. The data is recorded at such a high acquisition frequency that we can expect this corresponds very well with the lattice strain observed from the synchrotron diffraction patterns.

The `plot_thermomechanical_data` function in the `deformation` functions returns arrays of the true stress and true strain at the adjusted acquisition frequency of the recorded diffraction pattern images.

In [ ]:
true_stress, true_strain = deformation.plot_thermomechanical_data(thermomechanical_equipment,thermomechanical_file_path, 
                                                            output_file_path, experiment_number,
                                                            minimum_stress, deform_sequence, 
                                                            number_of_frames, acquisition_frequency_sxrd, 
                                                            filter_equipment)

## Load the SXRD inputs

The additional information to analyse the synchrotron data can be loaded from the **second** yaml file using the command below.

Most importantly this defines the azimuthal angle for the loading direction.

In [ ]:
beam_energy, azimuth_load_direction, filter_sxrd = sxrd.extract_sxrd_input(config_path_deformation)

## Set the colour and marker scheme

The colour and marker scheme can be set automatically using the `colour_assign` and `marker_assign` functions. These are based on the number of lattice planes included in the analysis.

The current default for the alpha colour type is *viridis*.

The current default for the beta colour type is *PuRd*.

In [ ]:
plane_colour = sxrd.colour_assign(peak_label)
plane_marker = sxrd.marker_assign(peak_label)

This line can be used to reduce the peak labels to plot just the important peaks.

*Note, the beta 211 peak may or may not be resolved, as it is often obscured by the adjacent alpha 10-13 peak.*

In [ ]:
peak_label_reduced = ['10-10','0002','10-11','11-20','110','200','220']

## Check the start of deformation

### Option 1 - Lattice microstrain

The first option is to look for the characteristic change in lattice microstrain, by alterning the relative start and end points.

**After you have found the correct start and end numbers, add these values to the `start_deformation` and `end_deformation` parameters in the second deformation yaml file, save the yaml file, then re-start the notebook from the top and re-run all of the cells.**

In [ ]:
# start_number = 10
# end_number = 2386

# sxrd.find_start_end_microstrain(start_number, end_number, image_numbers_sorted, peak_position_time,
#                                 peak_label_reduced, azimuth_load_direction, plane_colour, plane_marker)

### Option 2 - Synchronised thermomechanical analogue output

The second option is to look for the characteristic change in load response recorded by the thermomechanical equipment, by alterning the relative start and end points.

This is only possible if an anologue output from the thermomechanical equipment has already been synchronised with the synchrotron diffraction pattern images. This means a file exists that lists some of the thermomechanical deformation conditions (typically position, temperature and load), along with the corresponding image number. This is usually only possible with ETMT data recorded at Diamond Light Source.

To load the analogue data we need some additional information from the **second** yaml file.

In [ ]:
# analogue_data_file_path, \
# load_conversion, temperature_conversion, position_conversion = deformation.extract_analogue_input(config_path_deformation)

Then, alternate the relative start and end points to look for the characteristic change in load response recorded by the termomechanical equipment.

**After you have found the correct start and end numbers, add these values to the `start_deformation` and `end_deformation` parameters in the second deformation yaml file, save the yaml file, then re-start the notebook from the top and re-run all of the cells.**

In [ ]:
# start_number = 2059
# end_number = 2553

# deformation.plot_analogue_data(analogue_data_file_path, start_number, end_number, 
#                                load_conversion, temp_conversion, position_conversion)

## Analysing the peak changes

### Azimuthal peak position

Check how the minimum and maximum position of the ring changes throughout the test. These plots can be a bit messy as for uniaxial loading conditions the maxmimum and minimum positions of the peaks will alternate at ${180^\circ}$. However, they are useful to check that the the diffraction pattern ring asymmetry, and the individual positions of the peaks, are moving in a way that you'd expect given the loading conditions.

In [ ]:
sxrd.follow_azimuth_angle(peak_position_time, image_numbers_sorted,
                          start_deformation, end_deformation,
                          peak_label_reduced, data_resolution, 
                          plane_colour, plane_marker, 
                          output_file_path, experiment_number)

### Lattice microstrain behaviour

The thermomechanical test results can then be combined with the elastic lattice strain response measured from the synchrotron diffraction patterns. This shows us the relative strength of the different phases and different grain orientations with respect to the loading direction.

- **"Soft" grain orientations** - reach an elastic lattice strain limit with greater applied stress, and therefore can be assumed to have begun plastically deforming at a lower stress.

- **"Hard" grain orientations** - continue to take up greater elastic lattice strain with greater applied stress, these reach there elastic strain limit later, at higher stresses.

The `plot_microstrain_stress` function includes additional inputs defined below. 

The `number_of_points` provides a cut off to remove measurements after yield, which can obscure the elastic-plastic transition. 

The microstrain, true stress and true strain limits can be used to set the plot limits for comparison with other data sets.

In [ ]:
# alter this value to cut off any additional data, such as after plastic yield
number_of_points_stress = 3000
number_of_points_strain = 0

# set these values to zero if you do not want microstrain or true stress limits applied to the data
microstrain_limit = 0
true_stress_limit = 0
true_strain_limit = 0

In [ ]:
sxrd.plot_microstrain_stress(start_deformation, end_deformation, image_numbers_sorted,
                             peak_position_time, true_stress, 
                             peak_label_reduced, azimuth_load_direction, plane_marker, plane_colour, 
                             output_file_path, experiment_number, filter_sxrd,
                             number_of_points_stress, microstrain_limit, true_stress_limit)

The plot of microstrain variation over time with applied strain, shows us the different loading states of the different two-phase grain types. For instance, load relaxation at high temperature might be showing us preferential recrystallization or phase transformation in the different grain types.

In [ ]:
sxrd.plot_microstrain_strain(start_deformation, end_deformation, image_numbers_sorted,
                             peak_position_time, true_strain, 
                             peak_label_reduced, azimuth_load_direction, plane_marker, plane_colour, 
                             output_file_path, experiment_number, filter_sxrd,
                             number_of_points_strain, microstrain_limit, true_strain_limit)

In [ ]:
microstrain_limit = 17000
true_stress_limit = 1050
true_strain_limit = 0.07

sxrd.plot_microstrain_stress(start_deformation, end_deformation, image_numbers_sorted,
                             peak_position_time, true_stress, 
                             peak_label_reduced, azimuth_load_direction, plane_marker, plane_colour, 
                             output_file_path, experiment_number, filter_sxrd,
                             number_of_points_stress, microstrain_limit, true_stress_limit)

sxrd.plot_microstrain_strain(start_deformation, end_deformation, image_numbers_sorted,
                             peak_position_time, true_strain, 
                             peak_label_reduced, azimuth_load_direction, plane_marker, plane_colour, 
                             output_file_path, experiment_number, filter_sxrd,
                             number_of_points_strain, microstrain_limit, true_strain_limit)

### Peak intensity behaviour [TODO]

### Peak FWHM behaviour [TODO]

### Polar plot of lattice microstrain behaviour [TODO]